In [1]:
import os
import glob
import rasterio as rio
from rasterio.mask import raster_geometry_mask
from rasterio import plot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

In [2]:
data_dir = '../data/'

In [3]:
# Batch process

orig_imgs = glob.glob(data_dir +'images/*/SkySat/202*.tif')
orig_imgs

['../data/images/2022/SkySat/20220720_153112_ssc4d2_0013_analytic_SR_clipped.tif',
 '../data/images/2022/SkySat/20220707_151445_ssc4d2_0006_analytic_SR_clipped.tif',
 '../data/images/2022/SkySat/20220622_160829_ssc15d2_0014_analytic_SR_clipped.tif',
 '../data/images/2021/SkySat/20210726_160338_ssc1d2_0013_ar.tif',
 '../data/images/2021/SkySat/20210802_184019_ssc10d2_0005_ar.tif',
 '../data/images/2021/SkySat/20210707_150122_ssc19d2_0016_ar.tif',
 '../data/images/2021/SkySat/20210816_171007_ssc18d2_0015_ar.tif',
 '../data/images/2021/SkySat/20210809_185329_ssc9d2_0014_ar.tif',
 '../data/images/2020/SkySat/20200710_155028_ssc3d2_0012_ar.tif',
 '../data/images/2020/SkySat/20200812_153924_ssc3d2_0013_ar.tif',
 '../data/images/2020/SkySat/20200625_154704_ssc12d2_0013_ar.tif']

In [9]:
orig_imgs_ps = glob.glob(data_dir +'images/*/PScope/202*.tif')
orig_imgs_ps

['../data/images/2022/PScope/20220613_154958_91_240c_3B_AnalyticMS_SR_8b_clipped.tif',
 '../data/images/2022/PScope/20220630_150422_85_241d_3B_AnalyticMS_SR_8b_clipped.tif',
 '../data/images/2022/PScope/20220704_155057_15_240a_3B_AnalyticMS_SR_8b_clipped.tif',
 '../data/images/2022/PScope/20220726_155410_80_2407_3B_AnalyticMS_SR_8b_clipped.tif',
 '../data/images/2022/PScope/20220624_150204_01_241b_3B_AnalyticMS_SR_8b_clipped.tif',
 '../data/images/2022/PScope/20220731_153552_40_2461_3B_AnalyticMS_SR_8b_clipped.tif',
 '../data/images/2022/PScope/20220709_155032_68_240a_3B_AnalyticMS_SR_8b_clipped.tif',
 '../data/images/2021/PScope/20210808_150651_57_2460_3B_AnalyticMS_SR_8b_harmonized_clip.tif',
 '../data/images/2021/PScope/20210804_152058_95_222f_3B_AnalyticMS_SR_8b_harmonized_clip.tif',
 '../data/images/2021/PScope/20210624_160110_56_225b_3B_AnalyticMS_SR_8b_harmonized_clip.tif',
 '../data/images/2021/PScope/20210715_151038_58_2450_3B_AnalyticMS_SR_8b_harmonized_clip.tif',
 '../data/i

In [10]:
# Try normalizing by min/max values for each band
# test_img = orig_imgs[1]

# with rio.open(test_img) as src:
#     as_arr = src.read()
#     arr_min = as_arr.min(axis=(1, 2), keepdims=True)  # set axis to (1,2) to normalize for each band separately
#     arr_max = as_arr.max(axis=(1, 2), keepdims=True)
    
#     normalized = (as_arr - arr_min)/(arr_max-arr_min)
#     print(normalized.shape)

In [7]:
def minmax_norm(img_paths):
    '''
    Returns a normalized image for each input raster.
    Normalization scales reflectance values to 0-1.
    Each band is normalized separately.
    
    Input:
    img_paths - list of abs. paths to input images
    
    '''
    for i in img_paths:
        
        with rio.open(i) as src:
            as_arr = src.read()
            arr_min = as_arr.min(axis=(1, 2), keepdims=True)
            arr_max = as_arr.max(axis=(1, 2), keepdims=True)
            
            normalized = (as_arr - arr_min)/(arr_max-arr_min)
            
            kwargs = src.meta
            kwargs.update(dtype=rio.float32, count =4)
            
            with rio.open(str(os.path.split(i)[0])+'/NORM_minmax_'+str(os.path.basename(i)), 
                          'w', **kwargs) as dst:
                
                dst.write_band(1, normalized[0].astype(rio.float32))
                dst.write_band(2, normalized[1].astype(rio.float32))
                dst.write_band(3, normalized[2].astype(rio.float32))
                dst.write_band(4, normalized[3].astype(rio.float32))


In [11]:
def minmax_norm_ps(img_paths):
    '''
    Returns a normalized image for each input raster.
    Normalization scales reflectance values to 0-1.
    Each band is normalized separately.
    
    Input:
    img_paths - list of abs. paths to input images
    
    '''
    for i in img_paths:
        
        with rio.open(i) as src:
            as_arr = src.read()
            arr_min = as_arr.min(axis=(1, 2), keepdims=True)
            arr_max = as_arr.max(axis=(1, 2), keepdims=True)
            
            normalized = (as_arr - arr_min)/(arr_max-arr_min)
            
            kwargs = src.meta
            kwargs.update(dtype=rio.float32, count =8)
            
            with rio.open(str(os.path.split(i)[0])+'/NORM_minmax_'+str(os.path.basename(i)), 
                          'w', **kwargs) as dst:
                
                dst.write_band(1, normalized[0].astype(rio.float32))
                dst.write_band(2, normalized[1].astype(rio.float32))
                dst.write_band(3, normalized[2].astype(rio.float32))
                dst.write_band(4, normalized[3].astype(rio.float32))
                dst.write_band(5, normalized[4].astype(rio.float32))
                dst.write_band(6, normalized[5].astype(rio.float32))
                dst.write_band(7, normalized[6].astype(rio.float32))
                dst.write_band(8, normalized[7].astype(rio.float32))


In [12]:
minmax_norm_ps(orig_imgs_ps)

In [22]:
# Alternative normalization function using np.linalg.norm()


def get_norm_img(img_paths):
    '''
    Returns a normalized image for each input raster.
    Normalization is performed using the numpy.linalg.norm() function.
    The function calculates the Frobenius norm of each pixel in the input image
    and divides the pixel by the norm to return a unit vector.
    
    Input:
    img_paths - list of abs. paths to input images
    output_dir - directory to store normalized images
    
    '''
    for i in img_paths:
        with rio.open(i) as src:
            as_array = src.read()
            normalized = as_array/np.linalg.norm(as_array)
           
            
            kwargs = src.meta
            kwargs.update(dtype=rio.float32, count =4)
            
            with rio.open(str(os.path.split(i)[0])+'/NORM_'+str(os.path.basename(i)), 'w', **kwargs) as dst:
                dst.write_band(1, normalized[0].astype(rio.float32))
                dst.write_band(2, normalized[1].astype(rio.float32))
                dst.write_band(3, normalized[2].astype(rio.float32))
                dst.write_band(4, normalized[3].astype(rio.float32))

In [1]:
def get_norm_img_8band(img_paths):
    '''
    Returns a normalized image for each input raster.
    Normalization is performed using the numpy.linalg.norm() function.
    The function calculates the Frobenius norm of each pixel in the input image
    and divides the pixel by the norm to return a unit vector.
    
    Input:
    img_paths - list of abs. paths to input images
    output_dir - directory to store normalized images
    
    '''
    for i in img_paths:
        with rio.open(i) as src:
            as_array = src.read()
            normalized = as_array/np.linalg.norm(as_array)
            
            kwargs = src.meta
            kwargs.update(dtype=rio.float32, count =8)
            
            with rio.open(str(os.path.split(i)[0])+'/NORM_'+str(os.path.basename(i)), 'w', **kwargs) as dst:
                dst.write_band(1, normalized[0].astype(rio.float32))
                dst.write_band(2, normalized[1].astype(rio.float32))
                dst.write_band(3, normalized[2].astype(rio.float32))
                dst.write_band(4, normalized[3].astype(rio.float32))
                dst.write_band(5, normalized[4].astype(rio.float32))
                dst.write_band(6, normalized[5].astype(rio.float32))
                dst.write_band(7, normalized[6].astype(rio.float32))
                dst.write_band(8, normalized[7].astype(rio.float32))

In [23]:
#get_norm_img(orig_imgs)